# 2020/09/26 定期勉強会 データ分析演習

## ライブラリの読み込み
<br>
分析に使用するライブラリを読み込みます。ライブラリを読み込んでおくことで、できることが増えます。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as sp
from sklearn.datasets import load_boston

## データの読み込み
<br>
ライブラリを読み込んでいたおかげで、データを簡単に読み込めるようになりました。

In [ ]:
data = load_boston()

In [ ]:
# データを扱いやすいようにpandasのdata.frameに変換する
df = pd.DataFrame(data.data, columns = data.feature_names).assign(MEDV=np.array(data.target))

### データの軽い確認
<br>
データの頭5行何列か、という構造を軽く確認する。

In [ ]:
df.head()

### 変数の説明

| カラム | 説明 |
| ---- | ---- |
| CRIM | 町ごとの一人当たりの犯罪率 |
| ZN | 宅地の比率が25,000平方フィートを超える敷地に区画されている。 |
| INDUS |	町当たりの非小売業エーカーの割合 |
| CHAS | チャーリーズ川ダミー変数（川の境界にある場合は1、それ以外の場合は0） |
| NOX | 一酸化窒素濃度（1000万分の1） |
| RM | 1住戸あたりの平均部屋数 |
| AGE | 1940年以前に建設された所有占有ユニットの年齢比率 |
| DIS | 5つのボストンの雇用センターまでの加重距離 |
| RAD | ラジアルハイウェイへのアクセス可能性の指標 |
| TAX |	10,000ドルあたりの税全額固定資産税率 |
| PTRATIO |	生徒教師の比率 |
| B | 町における黒人の割合 |
| LSTAT | 人口当たり地位が低い率 |
| MEDV | 1000ドルでの所有者居住住宅の中央値 |

## 仮説の確認
<br>
仮説：住宅価格(\$)は、「敷地面積(feet$^2$)」、「建築年・改築年からの経過年数(年)」、「地上以上の部屋数(部屋)」によって算出される。

## 使用する変数
<br>
・目的変数（予測したい値）

SalePrice - 住宅価格
<br><br>

・説明変数（予測したい値に影響を与える変数）

LotArea - 敷地面積  

YearRemodAdd - 建築年・改築年

TotRmsAbvGrd - 地上以上の部屋数

YrSold - 販売年

## データ収集
<br>
仮説に沿ったデータのみを抽出する。

In [ ]:
train_data = train_data[['SalePrice', 'LotArea', 'YearRemodAdd', 'TotRmsAbvGrd', 'YrSold']]

### データの確認
<br>
変数の数とデータ件数の確認。

In [ ]:
print('(行数, 列数) = ', df.shape)

データの情報を確認する。

In [ ]:
df.info()

### 欠損値の確認

In [ ]:
print('Train columns with null values: {} \n' .format(df.isnull().sum()))

### データの基礎統計量の確認
<br>
・項目の説明

count: 要素の個数

mean: 平均

std: 標準偏差

min: 最小値

25%: 第一四分位数

75%: 第三四分位数

50%: 中央値 (median)

max: 最大値

In [ ]:
df.describe()

相関係数の算出

In [ ]:
df_corr = df_corr = df.corr()
display(df_corr)

相関係数を見やすく。

In [ ]:
sns.set_context('talk')
fig = plt.subplots(figsize = (15, 15))
 
sns.heatmap(df_corr, annot = True, fmt = '.2f', cmap = 'Blues', square = False)

全体のヒストグラムと散布図の描画。

In [ ]:
sns.pairplot(df)

### 目的変数のみ確認

ヒストグラム描画

In [ ]:
%matplotlib inline
df['MEDV'].hist(bins = 50, figsize = (10, 7.5))
plt.rcParams['figure.figsize'] = [20, 15]

箱ひげ図描画

In [ ]:
%matplotlib inline
fig_o, ax_o = plt.subplots(figsize=(9.6, 5))
ax_o.set_title('Box Plot')
ax_o.boxplot(df['MEDV'], vert = False,  labels=['MEDV'])
plt.rcParams['figure.figsize'] = [20, 15]

目的変数が正規分布に従っていないことを確認しておく。

### 説明変数のみ確認

ヒストグラム描画

In [ ]:
%matplotlib inline
train_data[['LotArea', 'YearRemodAdd', 'TotRmsAbvGrd', 'YrSold']].hist(bins=50, figsize=(15,7.5))
plt.rcParams['figure.figsize'] = [20, 15]

## データ加工

### 変数の作成
<br>
仮説では、「建築年・改築年からの経過年数(年)」が住宅価格に影響を与えているとしていたので、「経過年数」という変数を作成する。

それに伴い、元データの「建築年・改築年」、「販売年」という変数を削除する。

In [ ]:
train_data['ElapYr'] = train_data['YrSold'] - train_data['YearRemodAdd']
train_data.drop(['YearRemodAdd', 'YrSold'], axis = 1, inplace = True)

データの再確認

In [ ]:
train_data.head()

In [ ]:
train_corr = train_corr = train_data.corr()
display(train_corr)

In [ ]:
sns.set_context('talk')
fig = plt.subplots(figsize = (8, 8))
 
sns.heatmap(train_corr, annot = True, fmt = '.2f', cmap = 'Blues', square = True)

In [ ]:
sns.pairplot(train_data)

### 外れ値対処

In [ ]:
%matplotlib inline
fig_o, ax_o = plt.subplots(figsize=(9.6, 5))
ax_o.set_title('Box Plot')
ax_o.boxplot(train_data['LotArea'], vert = False,  labels=['LotArea'])
plt.rcParams['figure.figsize'] = [20, 15]

外れ値の除去。

In [ ]:
train_data.query('LotArea <= 39000', inplace = True)

In [ ]:
%matplotlib inline
train_data['LotArea'].hist(bins = 50, figsize = (10, 7.5))
plt.rcParams['figure.figsize'] = [20, 15]

In [ ]:
train_corr = train_corr = train_data.corr()
display(train_corr)

In [ ]:
sns.set_context('talk')
fig = plt.subplots(figsize = (8, 8))
 
sns.heatmap(train_corr, annot = True, fmt = '.2f', cmap = 'Blues', square = True)

「敷地面積」の外れ値を除去したら、「敷地面積」と「地上以上の部屋数」間に多重共線性(マルチコ)がありそうに見えてきましたが、今回は取り敢えず無視します。

In [ ]:
sns.pairplot(train_data)

## 回帰分析

一旦データを再確認する。

In [ ]:
train_data.head()

In [ ]:
train_data.info()

In [ ]:
train_data.describe()

In [ ]:
%matplotlib inline
np.log(train_data['SalePrice']).hist(bins = 50, figsize = (10, 7.5))
plt.rcParams['figure.figsize'] = [20, 15]

### モデル構築
<br>
・目的変数：

SalePrice - 住宅価格
<br><br>
・説明変数：

LotArea - 敷地面積

TotRmsAbvGrd - 地上以上の部屋数

ElapYr - 経過年数
<br><br>
#### モデル式
$ {\rm SalePrice} = \beta_0 + {\rm LotArea} \: \times \beta_1 + {\rm TotRmsAbvGrd} \: \times \beta_2 + {\rm ElapYr} \: \times \beta_3 $

In [ ]:
x = train_data[['LotArea', 'TotRmsAbvGrd', 'ElapYr']]
y = np.log(train_data['SalePrice'])
X = sm.add_constant(x)

model = sm.OLS(y, X)
result = model.fit()
result.summary()

### 着目する値
<br>
・Adj. R-squared: 自由度調整済み決定係数

・Prob (F-statistic): モデルに対するF検定の有意確率

・coef: 偏回帰係数

・P>|t|: 偏回帰係数に対するt検定の有意確率

・（必要に応じて）AIC: 赤池情報量基準

・（必要に応じて）BIC: ベイズ情報量基準

### 自社データに適応させる

In [ ]:
test_data = pd.read_csv('input/test.csv')
test_data = test_data[['LotArea', 'YearRemodAdd', 'TotRmsAbvGrd', 'YrSold']]
test_data['ElapYr'] = test_data['YrSold'] - test_data['YearRemodAdd']
test_data.drop(['YearRemodAdd', 'YrSold'], axis = 1, inplace = True)
test_data.query('LotArea <= 39000', inplace = True)

In [ ]:
test_data.head()

In [ ]:
pd.DataFrame(result.params)

In [ ]:
test_data['pred_val'] = result.params['const'] + \
    test_data['LotArea'] * result.params['LotArea'] + \
    test_data['TotRmsAbvGrd'] * result.params['TotRmsAbvGrd'] + \
    test_data['ElapYr'] * result.params['ElapYr']

In [ ]:
test_data['pred_val'] = np.exp(test_data['pred_val'])

### 予測された自社保有住宅価格

In [ ]:
pd.DataFrame(test_data['pred_val'])

## 影響度比較

In [ ]:
x = train_data[['LotArea', 'TotRmsAbvGrd', 'ElapYr']]
y = np.log(train_data['SalePrice'])
y.reset_index(inplace=True, drop=True)
x_n = pd.DataFrame(sp.stats.zscore(x, axis=0), \
                   columns=['LotArea', 'TotRmsAbvGrd', 'ElapYr'])
X_n = sm.add_constant(x_n)

model = sm.OLS(y, X_n)
result = model.fit()
result.summary()